In [1]:
import sys
import importlib

def add_path(path):
    if path not in sys.path:
        sys.path.append(path)

add_path('..')

import random

from reducto_examples import calc_lexer, llcalc, lrcalc
from reducto import lrk, llk, futils, grammar, parsing

import reducto_examples.calc_lexer as L

def reload():
    for module in [futils, grammar, llk, lrk, calc_lexer, llcalc, lrcalc, parsing]:
        importlib.reload(module)

reload()

# Functions

In [2]:
def display_symbol(symbol):
    return symbol.__name__

def display_seq(seq):
    return ' '.join([display_symbol(symbol) for symbol in seq])
               
def display_lritem(lritem):
    a = display_symbol(lritem.rule.left)
    b1 = display_seq(lritem.rule.right[:lritem.pos])
    b2 = display_seq(lritem.rule.right[lritem.pos:])
    fwd = display_seq(lritem.fwd)
    return f'{a} -> {b1} . {b2} ... {fwd}'

def display_lrset(lrset):
    res = [
        display_lritem(lritem)
        for lritem in lrset
    ]
    res.sort()
    return '\n'.join(res)

def display_rule(rule):
    a = display_symbol(rule.left)
    b = display_seq(rule.right)
    return f'{a} -> {b}'

def display_grammar(grammar):
    rules = [display_rule(rule) for rule in grammar.rules]
    return '\n'.join(sorted(rules))

In [3]:
reload()
tokens = parsing.TokenStream(calc_lexer.parse_str('1 + 2 * 3'))
tokens.forward_types(2)[0]

reducto_examples.calc_lexer.Number

# Adhoc

In [4]:
reload()
parsing.LookaheadInfo(llcalc.calc, 1).follow

{reducto_examples.llcalc.Top: frozenset({(),
            (reducto_examples.calc_lexer.RP,)}),
 reducto_examples.llcalc.TopTail: frozenset({(),
            (reducto_examples.calc_lexer.RP,)}),
 reducto_examples.llcalc.LeadSign: frozenset({(reducto_examples.calc_lexer.LP,),
            (reducto_examples.calc_lexer.Number,)}),
 reducto_examples.llcalc.MultTail: frozenset({(),
            (reducto_examples.calc_lexer.MinusSign,),
            (reducto_examples.calc_lexer.PlusSign,),
            (reducto_examples.calc_lexer.RP,)}),
 reducto_examples.llcalc.Mult: frozenset({(),
            (reducto_examples.calc_lexer.MinusSign,),
            (reducto_examples.calc_lexer.PlusSign,),
            (reducto_examples.calc_lexer.RP,)}),
 reducto_examples.llcalc.Sign: frozenset({(reducto_examples.calc_lexer.LP,),
            (reducto_examples.calc_lexer.Number,)}),
 reducto_examples.llcalc.LikeNumber: frozenset({(),
            (reducto_examples.calc_lexer.DivSign,),
            (reducto_examples.ca

# Recursive descent parser

In [5]:
def fwd(tokens):
    if tokens.forward(1):
        return type(tokens.forward(1)[0].value)
    return None

def error(tokens, msg):
    if not tokens.forward(1):
        pos = 'end of input'
    else:
        pos = tokens.forward(1)[0].posinfo[0]
    raise Exception(f'{msg} at {pos}')

def do_parse(method, str_to_parse, *args, **kwargs):
    try:
        return method(calc_lexer.stream_from_str(str_to_parse), *args, **kwargs)
    except Exception as exc:
        return str(exc)

In [6]:
stream = calc_lexer.stream_from_str('1+2*3')
while fwd(stream):
    print(fwd(stream), stream.shift())

<class 'reducto_examples.calc_lexer.Number'> TokenRecord(value=Number(value=1.0), posinfo=(0, 1))
<class 'reducto_examples.calc_lexer.PlusSign'> TokenRecord(value=PlusSign(value='+'), posinfo=(1, 2))
<class 'reducto_examples.calc_lexer.Number'> TokenRecord(value=Number(value=2.0), posinfo=(2, 3))
<class 'reducto_examples.calc_lexer.MultSign'> TokenRecord(value=MultSign(value='*'), posinfo=(3, 4))
<class 'reducto_examples.calc_lexer.Number'> TokenRecord(value=Number(value=3.0), posinfo=(4, 5))


In [7]:
def parse_sign(tokens, mandatory=True):
    if fwd(tokens) == L.PlusSign:
        tokens.shift()
        return 1
    elif fwd(tokens) is L.MinusSign:
        tokens.shift()
        return -1
    elif mandatory:
        raise error(tokens, 'plus or minus sign expected')
    return 1

assert do_parse(parse_sign, '+1') == 1
assert do_parse(parse_sign, '-1') == -1
assert do_parse(parse_sign, '(') == 'plus or minus sign expected at 0'
assert do_parse(parse_sign, '(', mandatory=False) == 1

In [8]:
def parse_top(tokens):
    res = parse_l1(tokens)
    if not fwd(tokens):
        raise error(tokens, 'unexpected input')
    return res

def parse_l1(tokens):
    s = parse_sign(tokens, mandatory=False)
    res = s * parse_l2(tokens)
    while fwd(tokens) and fwd(tokens) is not L.RP:
        s = parse_sign(tokens, mandatory=True)
        res = res + s * parse_l2(tokens)
    return res

def parse_l2(tokens):
    res = parse_numlike(tokens)
    while fwd(tokens) is L.MultSign or fwd(tokens) is L.DivSign:
        op = tokens.shift()
        val = parse_numlike(tokens)
        if isinstance(op.value, L.MultSign):
            res = res * val 
        elif isinstance(op.value, L.DivSign):
            res = res / val
    return res

def parse_numlike(tokens):
    if fwd(tokens) is L.Number:
        res = tokens.shift()
        return res.value.value
    elif fwd(tokens) is L.LP:
        tokens.shift()
        res = parse_l1(tokens)
        if fwd(tokens) is L.RP:
            tokens.shift()
            return res
        else:
            error(tokens, 'RP expected')

            
do_parse(parse_l1, '+1 + (-2 * 3 + 4) * 5 + 6')

-3.0

# Что такое грамматика

In [9]:
reload()
lrcalc.calc

In [10]:
def random_expand(grammar, seq, max_tries):
    nonterms, terms = grammar.symbols()
    res = [seq]
    k = 0
    while len(res) < max_tries:
        seq = res[-1]
        k += 1
        expanded = False
        for num in range(len(seq)):
            if seq[num] not in nonterms:
                continue
            expanded = True
            rules = [
                rule 
                for rule in grammar.rules
                if rule.left == seq[num]
            ]
            rule = random.choice(rules)
            newseq = seq[0:num] + rule.right + seq[num+1:]
            res.append(newseq)
            break
        if not expanded:
            return res
    return res

seqs = random_expand(lrcalc.calc, (lrcalc.calc.start, ), 10)
for seq in seqs:
    print(' '.join([val.__name__ for val in seq]))

TopExp
TopExp MinusSign MultExp
MultExp MinusSign MultExp
LikeNumber MinusSign MultExp
LP TopExp RP MinusSign MultExp
LP MultExp RP MinusSign MultExp
LP LikeNumber RP MinusSign MultExp
LP Number RP MinusSign MultExp
LP Number RP MinusSign MultExp MultSign LikeNumber
LP Number RP MinusSign MultExp MultSign LikeNumber MultSign LikeNumber


## LL(1)

In [11]:
llcalc.calc

In [12]:
reload()
str_to_parse = '+1 + (-2 * 3 + 4) * 5 + 6'

llinfo = llk.LLInfo(llcalc.calc, 1)
llparser = llk.LLParser(
    k=1, 
    start=llcalc.calc.start, 
    terms=llinfo.terms,
    rule_by_fwd=llinfo.rule_by_fwd
)

llparser.parse(calc_lexer.stream_from_str(str_to_parse))

Top(value=-3.0)

# LR(1)

In [13]:
reload()

lrinfo = lrk.LRInfo(lrcalc.calc, 1)

def display_action(action):
    if isinstance(action, lrk.Shift):
        a = display_symbol(action.rule.left)
        b1 = display_seq(action.rule.right[:action.pos])
        b2 = display_seq(action.rule.right[action.pos:])
        return f'shift {a} -> {b1} . {b2}'
    if isinstance(action, lrk.Reduce):
        return f'reduce {display_seq(action.rule.right)} to {display_symbol(action.rule.left)}'

for prefix, fwd, actions in lrinfo.conflicts:
    print(display_seq(prefix), '|', display_seq(fwd))
    for action in actions:
        print('    ', display_action(action))

In [14]:
reload()
str_to_parse = '+1 + (-2 * 3 + 4) * 5 + 6'


lrinfo = lrk.LRInfo(lrcalc.calc, 1)
lrparser = lrk.LRParser(
    k=1, 
    start=lrcalc.calc.start, 
    lr_tables=lrinfo.lr_tables
)

lrparser.parse(calc_lexer.stream_from_str(str_to_parse))

  ...  PlusSign
PlusSign +  ...  Number
PlusSign + Number 1.0  ...  PlusSign
PlusSign + LikeNumber 1.0  ...  PlusSign
PlusSign + MultExp 1.0  ...  PlusSign
TopExp 1.0  ...  PlusSign
TopExp 1.0 PlusSign +  ...  LP
TopExp 1.0 PlusSign + LP (  ...  MinusSign
TopExp 1.0 PlusSign + LP ( MinusSign -  ...  Number
TopExp 1.0 PlusSign + LP ( MinusSign - Number 2.0  ...  MultSign
TopExp 1.0 PlusSign + LP ( MinusSign - LikeNumber 2.0  ...  MultSign
TopExp 1.0 PlusSign + LP ( MinusSign - MultExp 2.0  ...  MultSign
TopExp 1.0 PlusSign + LP ( MinusSign - MultExp 2.0 MultSign *  ...  Number
TopExp 1.0 PlusSign + LP ( MinusSign - MultExp 2.0 MultSign * Number 3.0  ...  PlusSign
TopExp 1.0 PlusSign + LP ( MinusSign - MultExp 2.0 MultSign * LikeNumber 3.0  ...  PlusSign
TopExp 1.0 PlusSign + LP ( MinusSign - MultExp 6.0  ...  PlusSign
TopExp 1.0 PlusSign + LP ( TopExp -6.0  ...  PlusSign
TopExp 1.0 PlusSign + LP ( TopExp -6.0 PlusSign +  ...  Number
TopExp 1.0 PlusSign + LP ( TopExp -6.0 PlusSign + Numb

TopExp(value=-3.0)